In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation
from tensorflow.python.keras import metrics
from sklearn.preprocessing import StandardScaler
from tensorflow.python import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk

# 파일읽기

In [2]:
df=pd.read_excel('../optimal_data3_1/'+'Continous_2weeks_2day_1term.xlsx')

In [3]:
df

,0,1,2,3,4,5,6,7,8
0,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
1,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
2,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
3,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
4,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
...,...,...,...,...,...,...,...,...,...
58820,#ZZLSSL,68.00,29.048656,0.0,0.0,0.0,0.0,153.0,0
58821,#ZZLSSL,68.00,29.048656,0.0,0.0,0.0,0.0,153.0,0
58822,#ZZLSSL,68.00,29.048656,0.0,0.0,0.0,0.0,153.0,0
58823,#ZZLSSL,68.00,29.048656,0.0,0.0,0.0,0.0,153.0,0


# 데이터를 필요한 부분으로 X, Y 나눠 읽어오기

In [4]:
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]
CMResult=[[[0 for k in range(3)]for j in range(3)] for i in range(10)]
pred_list=[]
LossResult=[[0 for j in range(100)] for i in range(10)]
AccuracyResult=[[0 for j in range(100)] for i in range(10)]

In [5]:
#한 사람당 데이터 수
Count_1=int(181*0.1)
#한 사람당 데이터 수
Count_2=181

X_test=pd.DataFrame()
X_train=pd.DataFrame()
y_test=pd.DataFrame()
y_train=pd.DataFrame()
empty=pd.DataFrame()

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]

# 모델

## Deep Neural Network

## Input Feature : 6

## Hidden Layer : 128 - 256 - 128

## optimizer : adam, loss : categorical_crossentropy

In [6]:
for k in range(10):
    globals()["model{}".format(k)] = keras.Sequential([
        keras.layers.Flatten(input_shape=(6,)),
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dense(256, activation=tf.nn.relu),
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dense(3, activation=tf.nn.softmax),
    ])
    
    if k==0:
        model0.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    if k==1:
        model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
        
    if k==2:
        model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    if k==3:
        model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    if k==4:
        model4.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    if k==5:
        model5.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    if k==6:
        model6.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    if k==7:
        model7.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    if k==8:
        model8.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    if k==9:
        model9.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer='adam',
                      metrics=['accuracy'])
    

In [ ]:
for i in range(10):
    X_test=empty
    X_train=empty
    y_test=empty
    y_train=empty
    y_test_list=[]
    
    for j in range(325):
        X_temp_test=X.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        X_test=pd.concat([X_test,X_temp_test])
        X_temp_train=X.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        X_train=pd.concat([X_train,X_temp_train])
        
        y_temp_test=y.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        y_test=pd.concat([y_test,y_temp_test])
        y_temp_train=y.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        y_train=pd.concat([y_train,y_temp_train])
        
    
#     print('SMOTE 적용 전 Train 레이블 값 분포: \n', y_train.value_counts())
#     print('SMOTE 적용 전 Test 레이블 값 분포: \n', y_test.value_counts())
    
    # SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
#     print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_test.shape, y_test.shape)
#     print('SMOTE 적용 후 Train 레이블 값 분포: \n', y_train.value_counts())
#     print('SMOTE 적용 후 Test 레이블 값 분포: \n', y_test.value_counts())
    
    # 원핫인코딩
    # 예시 : 1 , 2 -> (1,0) , (0,1)
    y_train=pd.get_dummies(y_train[0])
    y_test=pd.get_dummies(y_test[0])
    

    if i==0:
        history=model0.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model0.predict(X_test)
        results = model0.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   

        
    if i==1:
        history=model1.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model1.predict(X_test)
        results = model1.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   

    if i==2:
        history=model2.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model2.predict(X_test)
        results = model2.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   
    
    if i==3:
        history=model3.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model3.predict(X_test)
        results = model3.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   
    
    if i==4:
        history=model4.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model4.predict(X_test)
        results = model4.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   
    
    if i==5:
        history=model5.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model5.predict(X_test)
        results = model5.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   
    
    if i==6:
        history=model6.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model6.predict(X_test)
        results = model6.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   
    
    if i==7:
        history=model7.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model7.predict(X_test)
        results = model7.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   
    
    if i==8:
        history=model8.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model8.predict(X_test)
        results = model8.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))   
    
    if i==9:
        history=model9.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
        predicted=model9.predict(X_test)
        results = model9.evaluate(X_test, y_test, batch_size=18)
        print("[{}]Accuracy : {}".format(i,results[1]))
    
    predicted=pd.DataFrame(predicted)
    predicted=predicted.idxmax(axis=1)
    y_test=y_test.idxmax(axis=1)
    
#     print("predicted",predicted)
#     print("y_test",y_test)
    
    #f1score
    f1 = f1_score(y_test,predicted, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    p_rlist=sk(y_test,predicted,average='weighted')
    print("[{}]Precision : {}".format(i,p_rlist[0]))
    print("[{}]Recall : {}".format(i,p_rlist[1]))
    print()
     #결과 배열에 넣기
    Result[i][0]=results[1]
    Result[i][1]=f1
    Result[i][2]=p_rlist[0]
    Result[i][3]=p_rlist[1]
    del results
    del f1
    del p_rlist
    
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    import matplotlib.pyplot as plt

    cm = confusion_matrix(y_test, predicted, labels=[0,1,2],normalize="true")
    CMResult[i]=cm
    print(CMResult[i])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1,2])
    disp.plot()
    filename=("ConfusionMatrix_"+str(i))
    plt.savefig(filename+".png")
    plt.show()
    
    LossResult[i]=history.history['loss']
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper right')
    filename=("LossGraph_"+str(i))
    plt.savefig(filename+".png")
    plt.show()
    
    AccuracyResult[i]=history.history['accuracy']
    plt.plot(history.history['accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper right')
    filename=("AccuracyGraph_"+str(i))
    plt.savefig(filename+".png")
    plt.show()
    
    del filename

Epoch 1/100
8398/8398 [==============================] - 29s 3ms/step - loss: 0.8096 - accuracy: 0.6133 - val_loss: 1.3613 - val_accuracy: 0.4497
Epoch 2/100
8398/8398 [==============================] - 27s 3ms/step - loss: 0.5990 - accuracy: 0.7439 - val_loss: 1.6010 - val_accuracy: 0.4662
Epoch 3/100
8398/8398 [==============================] - 27s 3ms/step - loss: 0.5122 - accuracy: 0.7886 - val_loss: 1.8795 - val_accuracy: 0.4684
Epoch 4/100
8398/8398 [==============================] - 29s 3ms/step - loss: 0.4633 - accuracy: 0.8117 - val_loss: 2.1134 - val_accuracy: 0.4688
Epoch 5/100
8398/8398 [==============================] - 29s 3ms/step - loss: 0.4286 - accuracy: 0.8273 - val_loss: 2.3012 - val_accuracy: 0.4665
Epoch 6/100
8398/8398 [==============================] - 27s 3ms/step - loss: 0.4036 - accuracy: 0.8387 - val_loss: 2.4390 - val_accuracy: 0.4568
Epoch 7/100
8398/8398 [==============================] - 28s 3ms/step - loss: 0.3833 - accuracy: 0.8473 - val_loss: 2.5295 -

In [ ]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

In [ ]:
print("Average of Accuracy {}".format(Result_df['Accuracy'].mean()))
print("Average of F1-Score {}".format(Result_df['F1-Score'].mean()))
print("Average of Precision {}".format(Result_df['Precision'].mean()))
print("Average of Recall {}".format(Result_df['Recall'].mean()))

In [ ]:
Matrix=pd.DataFrame(Result_df['Accuracy'],columns=['Accuracy'])
Matrix['Accuracy']=Result_df['Accuracy']
A=[Result_df['Accuracy'].mean(),Result_df['F1-Score'].mean(),Result_df['Precision'].mean(),Result_df['Recall'].mean()]
A=pd.DataFrame(A,columns=['Accuracy'])
Matrix=pd.concat([Matrix,A])
Matrix=Matrix.transpose()
Matrix.to_excel('./PFMatrix_Keras2.xlsx')

In [ ]:
AverageLossResult=[0 for j in range(100)]
AverageAccuracyResult=[0 for j in range(100)]

for i in range(10):
    for j in range(100):
#         print(i,j)
        AverageLossResult[j]=AverageLossResult[j]+LossResult[i][j]
        AverageAccuracyResult[j]=AverageAccuracyResult[j]+AccuracyResult[i][j]
for k in range(100):
    AverageLossResult[k]=AverageLossResult[k]/10
    AverageAccuracyResult[k]=AverageAccuracyResult[k]/10

In [ ]:
    plt.plot(AverageLossResult)
    plt.title('Train Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper right')
    filename=("LossGraph")
    plt.savefig(filename+".png")
    plt.show()
    
    plt.plot(AverageAccuracyResult)
    plt.title('Train Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper right')
    filename=("AccuracyGraph")
    plt.savefig(filename+".png")
    plt.show()

In [ ]:
import numpy as np

AverageCM=[[0 for i in range(3)] for j in range(3)]
for i in range(10):
    for j in range(3):
        for k in range(3):
            AverageCM[j][k]=AverageCM[j][k]+CMResult[i][j][k]

for j in range(3):
        for k in range(3):
            AverageCM[j][k]=AverageCM[j][k]/10
            
AverageCM=np.array(AverageCM)           
disp = ConfusionMatrixDisplay(confusion_matrix=AverageCM,display_labels=[0,1,2])
disp.plot()
filename=("DNN_ConfusionMatrix")
plt.savefig(filename+".png")
plt.show()